In [2]:
from download_day import download_day
from load_saved_fd4 import load_saved_fd4
from scrape_aircraft_list import scrape_aircraft_list
from get_perf_model_typecodes import get_perf_model_typecodes 
from match_icao_model import match_icao_model
from process_airport_list import process_airport_list
import pandas as pd
import numpy as np
from time import sleep
from geopy import distance

In [3]:
# User Inputs:
start_time_str       = '2024-01-01T00:00:00Z'
stop_time_str        = '2024-12-31T23:59:59Z'
query_limit          = 30e4
send_notification    = True
make_plot            = False
output_dir           = "/scratch/omg28/Data/no_track_2024"

# Convert start and stop times to datetime objects
start_time_simple = pd.to_datetime(start_time_str).strftime("%Y-%m-%d")
stop_time_simple = pd.to_datetime(stop_time_str).strftime("%Y-%m-%d")

Section 1. Download flight information from Opensky

In [ ]:
"""
# Download data from OpenSky history database
# WARNING: This may take a long time to run and will use a lot of disk space. Recommend running this in a .py script to avoid recurring memory issues found in Jupyter notebooks.
download_day(start_time_str, stop_time_str, query_limit, send_notification, make_plot, output_dir)
"""

In [ ]:
# load the saved day of data from a .pkl file to a pandas dataframe
loaded_day = load_saved_fd4(stop_time_str, output_dir, query_limit)

Section 2. Scrape the EUCONTROL database for aircraft flight performance information

In [ ]:
"""
# scrape the list of aircraft with performance models from the EUCONTROL website
perf_model_typecodes = get_perf_model_typecodes()
## WARNING: THIS CAN GET YOUR IP ADDRESS BLOCKED IF YOU RUN IT REPEATEDLY. IT IS A WEB SCRAPER.
aircraft_list = scrape_aircraft_list(perf_model_typecodes)

"""

Section 3. Process the aircraft performance and flight data, retaining only flights we have takeoff, landing, plane type, and aircraft performance model information about. Further, drop all flights with aircraft codes that do not correspond to a code in the ourairports registry.

In [ ]:
# process the loaded time period of data. - 20s
# - matches 24 bit transponder codes from Feb 2025 aircraft metadata database with the ICAO typecode
# - removes all flights for which the transponder code does not have a corresponding ICAO typecode
# - removes all flights for which the takeoff OR landing airport is unknown
flights_with_typecode = match_icao_model(start_time_str, stop_time_str, query_limit)


There are 3183 unique typecodes in the aircraft database.
Saved result_df to /scratch/omg28/Data/aircraftdb/2024-01-01_to_2024-12-31_300000_typecodes_added.pkl


In [5]:
# Load in the list of ICAO typecodes that have a performance model
flights_with_typecode = pd.read_pickle('/scratch/omg28/Data/aircraftdb/2024-01-01_to_2024-12-31_300000.pkl')
# load in the list of all flights with typecode aand takeoff/landing airport from 2024
performance_model_typecodes = pd.read_pickle('/scratch/omg28/Data/aircraftdb/performance_models_typecodes.pkl')

# get info before the merge
print("Before merging:")
print(flights_with_typecode.info())

# inner join the loaded year of data with the scraped aircraft list
# - this will remove all flights for which the ICAO typecode does not have a corresponding performance model
flights_with_perf_typecode = pd.merge(flights_with_typecode, performance_model_typecodes, how='inner', on = 'typecode')

print("After merging:")
print(flights_with_perf_typecode.info())


Before merging:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24461374 entries, 0 to 24461373
Data columns (total 4 columns):
 #   Column               Dtype          
---  ------               -----          
 0   icao24               object         
 1   estdepartureairport  string[pyarrow]
 2   estarrivalairport    string[pyarrow]
 3   typecode             object         
dtypes: object(2), string[pyarrow](2)
memory usage: 746.5+ MB
None
After merging:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20257579 entries, 0 to 20257578
Data columns (total 4 columns):
 #   Column               Dtype          
---  ------               -----          
 0   icao24               object         
 1   estdepartureairport  string[pyarrow]
 2   estarrivalairport    string[pyarrow]
 3   typecode             object         
dtypes: object(2), string[pyarrow](2)
memory usage: 623.0+ MB
None


In [9]:
# print a list of typecodes in complete_flights_perf_model that do not have a performance model
missing_perf_models = flights_with_typecode[~flights_with_typecode['typecode'].isin(performance_model_typecodes['typecode'])]
print("Aircraft models with observed flights, but no performance model: " + str(len(missing_perf_models['typecode'].unique())))
print("total flights in 2024 with TOLD and typecode, but no performance model: " + str(len(missing_perf_models)))


Aircraft models with observed flights, but no performance model: 1177
total flights in 2024 with TOLD and typecode, but no performance model: 4203795


In [10]:

# print a list of typecodes in performance_model_typecodes that do not correspond to a flight
missing_flights = performance_model_typecodes[~performance_model_typecodes['typecode'].isin(flights_with_typecode['typecode'])]
print("Number of performance models with no flights: " + str(len(missing_flights['typecode'].unique())))
print("Number of flights with TOLD, typecode, and performance model: " + str(len(flights_with_perf_typecode)))


Number of performance models with no flights: 89
Number of flights with TOLD, typecode, and performance model: 20257579


In [11]:
# count all flights starting and ending at the same airport
num_flights_no_dist = len(flights_with_perf_typecode[flights_with_perf_typecode['estdepartureairport'] == flights_with_perf_typecode['estarrivalairport']])
print("Number of flights with no distance: " + str(num_flights_no_dist))

# remove all flights starting or ending at the same airport
flights_with_perf_typecode_dist = flights_with_perf_typecode[flights_with_perf_typecode['estdepartureairport'] != flights_with_perf_typecode['estarrivalairport']]
print("Number of flights with TOLD, typecode, performance model, and nonzero distance: " + str(len(flights_with_perf_typecode)))


Number of flights with no distance: 2744283
Number of flights with TOLD, typecode, performance model, and nonzero distance: 20257579


In [ ]:
# load the takeoff and landing airport location information to the dataframe.
all_airports = process_airport_list()

# add the estimated departure airport location information to the dataframe
# - this will remove all flights for which the departure airport is unknown
flights_with_perf_typecode_dist_dep = pd.merge(flights_with_perf_typecode_dist, all_airports, how='inner', left_on = 'estdepartureairport', right_on = 'ident')
flights_with_perf_typecode_dist_dep.rename(columns = {'latitude_deg': 'estdeparturelat', 'longitude_deg': 'estdeparturelong', 'elevation_ft': 'estdeparturealt_ft'}, inplace=True)
flights_with_perf_typecode_dist_dep.drop(columns = ['ident', 'gps_code'], inplace=True)
# count the number of flights with no departure airport in the database
print("Number of flights with no departure airport location info in database: " + str(len(flights_with_perf_typecode_dist) - len(flights_with_perf_typecode_dist_dep)))

# add the estimated arrival airport location information to the dataframe
all_flights_filtered = pd.merge(flights_with_perf_typecode_dist_dep, all_airports, how='inner', left_on = 'estarrivalairport', right_on = 'ident')
all_flights_filtered.rename(columns = {'latitude_deg': 'estarrivallat', 'longitude_deg': 'estarrivallong', 'elevation_ft': 'estarrivalalt_ft'}, inplace=True)
all_flights_filtered.drop(columns = ['ident', 'gps_code', 'id_x', 'id_y'], inplace=True)
# count the number of flights with no arrival airport in the database
print("Number of flights with no arrival airport location info in database: " + str(len(flights_with_perf_typecode_dist_dep) - len(all_flights_filtered)))
print("Number of flights with TOLD, typecode, performance model, distance, and departure/arrival airport info: " + str(len(all_flights_filtered)))


Saved processed airports to /scratch/omg28/Data/airportdb/processed_airports.csv
Number of flights with no departure airport location info in database: 294131
Number of flights with no arrival airport location info in database: 427630
Number of flights with TOLD, typecode, performance model, distance, and departure/arrival airport info: 16791535


Section 4. Using the great circle distance, calculate the distance between the airport takeoff and landing.

In [25]:
# calculate the great circle distance between the departure and arrival airports - 4 min
all_flights_filtered['gc_km'] = all_flights_filtered.apply(lambda x: distance.great_circle((x['estdeparturelat'], x['estdeparturelong']), (x['estarrivallat'], x['estarrivallong'])).km, axis=1)
# adjust great circle distance using eq. 6 of seymour, et al (2020)
# all_flights_filtered = pd.read_pickle(f'/scratch/omg28/Data/no_track_2024/{start_time_simple}_to_{stop_time_simple}_filtered.pkl')

In [ ]:
# could be sped up by using a vectorized function # 1 min
all_flights_filtered['gc_FEAT_km'] = all_flights_filtered.apply(lambda x: 1.0387 * x['gc_km'] + 40.5, axis = 1)

In [28]:
# save the merged dataframe to a .pkl file
all_flights_filtered.to_pickle(f'/scratch/omg28/Data/no_track2024/{start_time_simple}_to_{stop_time_simple}_filtered.pkl')

In [30]:
print(all_flights_filtered.info())
print(all_flights_filtered.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16791535 entries, 0 to 16791534
Data columns (total 12 columns):
 #   Column               Dtype  
---  ------               -----  
 0   icao24               object 
 1   estdepartureairport  object 
 2   estarrivalairport    object 
 3   typecode             object 
 4   estdeparturelat      float64
 5   estdeparturelong     float64
 6   estdeparturealt_ft   float64
 7   estarrivallat        float64
 8   estarrivallong       float64
 9   estarrivalalt_ft     float64
 10  gc_km                float64
 11  gc_FEAT_km           float64
dtypes: float64(8), object(4)
memory usage: 1.5+ GB
None
   icao24 estdepartureairport estarrivalairport typecode  estdeparturelat  \
0  a01f01                KPIE              KPTK     C680        27.910200   
1  a08174                GA20              KCRG     C182        30.811100   
2  a08174                KCRG              K09J     C182        30.336300   
3  a146f9                KBOS              K

In [32]:
# remove all flights under 200 km:
all_flights_filtered = all_flights_filtered[all_flights_filtered['gc_FEAT_km'] > 200]
print("Number of flights with TOLD, typecode, performance model, distance, and departure/arrival airport info over 200 km: " + str(len(all_flights_filtered)))

Number of flights with TOLD, typecode, performance model, distance, and departure/arrival airport info over 200 km: 14021999


Section 5: generate flight profiles using openAP for flight database